## Load historical IV

In [305]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker('FL')  #PRX.AS ASML.AS
df = ticker.history(period='1D', interval="1d")
ticker.info["longName"]

'Foot Locker, Inc.'

In [306]:
price = df["Close"][0]

In [307]:
shares = ticker.info["sharesOutstanding"]
shares

94510304

In [308]:
ticker.balance_sheet.T["Cash"].iloc[1]

1680000000.0

## Financials

In [309]:
cash = ticker.balance_sheet.T["Cash"].iloc[0]
if math.isnan(cash):
    cash = ticker.balance_sheet.T["Cash"].iloc[1]
cash

804000000.0

In [310]:
netincome_df = ticker.cashflow.T[["Net Income"]]
netincome_df = netincome_df[netincome_df["Net Income"] > 0]
netincome_df = netincome_df.sort_values("")
netincome_df = netincome_df.reset_index()
netincome_df

,,Net Income
0,2019-02-02,541000000.0
1,2020-02-01,491000000.0
2,2021-01-30,323000000.0
3,2022-01-29,893000000.0


In [311]:
netincome_df["Growth"] = (netincome_df["Net Income"] - netincome_df["Net Income"].shift())/netincome_df["Net Income"]
netincome_df = netincome_df.fillna(0).iloc[1:, :][["Net Income", "Growth"]]
netincome_df

,Net Income,Growth
1,491000000.0,-0.101833
2,323000000.0,-0.520124
3,893000000.0,0.638298


## Params

In [312]:
import math

discount_rate = 0.10
years = 50
maturity = 15 #math.floor(years/2)
growth = netincome_df["Growth"].mean(axis=0)
if growth >= 0.3:
    growth = 0.3
growth_slowdown = growth/2

In [313]:
growth

0.005447013147039066

## Add forcast

In [314]:
for i in range(years):
    row ={"Net Income": netincome_df["Net Income"].iloc[-1] * (growth+1), "Growth": growth}
    rowdf = pd.DataFrame.from_records([row])
    netincome_df = pd.concat([netincome_df, rowdf], ignore_index=True, axis=0)
    
    if i == maturity:
        growth = -growth_slowdown

netincome_df = netincome_df.iloc[3:, :]

In [315]:
cashflows = netincome_df["Net Income"].to_list()

## DCF

In [316]:
import numpy_financial as npf

npv = npf.npv(discount_rate, cashflows);
value = npv + cash
forcast_price = value/shares
forcast_price

115.89136752744699

## DDM

In [329]:
exp_future_div = ticker.info["dividendRate"]
median_growth = 0.03
risk_free_rate = 0.03
mkt_return = .08
MKT_Risk_prem = mkt_return - risk_free_rate
beta = ticker.info["beta"]
COE = round(beta * MKT_Risk_prem + risk_free_rate,4)
COE

0.0967

In [330]:
fair_sharePrice = round(exp_future_div/(COE-median_growth),2)
fair_sharePrice

23.99

In [331]:
ddm = 0
if "dividendRate" in ticker.info and ticker.info["dividendRate"] != None and "beta" in ticker.info:
    exp_future_div = ticker.info["dividendRate"]
    div_growth = median_growth
    MKT_Risk_prem = mkt_return - risk_free_rate
    beta = ticker.info["beta"]
    COE = round(beta * MKT_Risk_prem + risk_free_rate,4)
    ddm = round(exp_future_div/(COE-div_growth),2)
print(ddm)

23.99
